# W1D3

# Generalized Linear Models

## Intro

some algorithms you might know, such as linear regression, logistic regression, and Poisson regression, are all just special cases of GLMs. They are very useful if you want to study encoding (i.e., predict neural activity from external covariates) or decoding (i.e., predict a behavior or intention from neural activity).